In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
import time
import datetime
# import gurobipy as gp  # import the installed package

In [ ]:
# U_LT = {}  
# U_T = {}  
# U_L = {}  
# V = {}   
# H = {}  
# D = {}  
# D_comp = {}  
# F = {}  
# C = {}  
# B = {} 
# config = {
#     'M': 1000000,
#     'lambda_G': 0.5,
#     'i_amount' :2, #2個客戶
#     'j_amount' :3, #三個設廠地點
#     'k_amount' :2, #兩種可分配資源（大車, 小車)
#     'l_amount' :0, #對手數量
#     'U_LT': [[3,5], [3,5], [3,5]],
#     'U_T': [5,10],
#     'U_L': [6,6,6],
#     'V': [[10,2], [2,1], [0,2]],
#     'H': [100, 50],
#     'D': [[1,1,1],[1,1,1]],
#     'D_comp':[],
#     'A_opponent_bar': [], #對手吸引力
#     'F': [0, 0, 0],
#     'C': [1, 1, 1], 
#     'B': [[1,1,1],[1,1,1],[1,1,1]],
# }

In [ ]:
'''
Instance Parameters
===============================
Customer Location: I
Facility Location: J
Resource Type: K
Competitor Facility Location: L

U_LT : Max amount of resource k allowed to be allocated to facility j, Dim : (j, k)
U_T : Max allocation limit for resource k, Dim : (k)
U_L : Max capacity for facility j, Dim : (j)
V : Attractiveness yield of resource k allocated to facility j, Dim : (j, k)
H : Max demand for customer i, Dim : (i)
D : Distance between customer i and facility j , Dim : (i, j)
D_comp : Distance between customer i and competitor l, Dim : (i, l)
F : Fixed cost for building facility j, Dim : (j)
C : Cost per unit of extra attractiveness at facility j, Dim : (j)
B : Cost of allocating a unit of resource k to facility j, Dim : (j, k)
===============================
'''

config = {
    'M': 1000000,
    'lambda_G': 0.5,
    'i_amount' :1, #客戶
    'j_amount' :1, #設廠地點
    'k_amount' :1, #可分配資源（大車, 小車)
    'l_amount' :1, #對手數量
    'U_LT': [[3]],
    'U_T': [5],
    'U_L': [4],
    'V': [[10]],
    'H': [100],
    'D': [[1]],
    'D_comp':[[1]],
    'A_opponent_bar': [0], #對手吸引力
    'F': [0],
    'C': [10], 
    'B': [[0]],
}

I = set(range(0, config['i_amount']))  
J = set(range(0, config['j_amount']))  
K = set(range(0, config['k_amount']))  
L = set(range(0, config['l_amount']))  

In [ ]:
# Initialize the model
model = Model("Competitive Facility Location")


'''
Decision Variables
===============================
y : Whether location j build facility or not, Dim: (j) , BINARY
x : The amount of resource k allocated to location j, Dim: (j,k), CONTINUOUS
aX : Extra attractiveness allocated to location j, Dim: (j), CONTINUOUS
===============================

Other Variables
u : Utility of facility j to customer point i, Dim:(i, j), CONTINUOUS
TA : Total Attractiveness for customer location i, Dim : (i), CONTINUOUS
P : Probability of customer i choosing facility j, Dim : (i, j), CONTINUOUS
===============================
'''
y = model.addVars(J, vtype=GRB.BINARY, name="y") #3.14
x = model.addVars(J, K, vtype=GRB.CONTINUOUS, name="x", lb=0) # 3.12
aX = model.addVars(J, vtype=GRB.CONTINUOUS, name="aX", lb=0) # 3.13

u = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="x", lb=0) # utility var(3.5)
P = model.addVars(I, J, vtype=GRB.CONTINUOUS, name="x", lb=0) # Probability of i choosing j
TA = model.addVars(I, vtype=GRB.CONTINUOUS, name="TA")
TAi_lambda = model.addVars(I, vtype=GRB.CONTINUOUS, name="TAi*lambda") #-lambda*TAi
G_exp_vars = model.addVars(I, vtype=GRB.CONTINUOUS, name="G_exp_vars") #e^-(lambda*TAi)
P_TA_ratio = model.addVars(I, vtype=GRB.CONTINUOUS, name="P_TA_ratio") #(3.10)的右半側

model.addConstrs((x[j, k] <= config['U_LT'][j][k] * y[j] for j in J for k in K), "(3.1) Resource only for built facility, and amount of type k for facility j is limit at U_LT[j, k]")
model.addConstrs((quicksum(x[j, k] for j in J) <= config['U_T'][k] for k in K), "(3.2) The total amount of resource k is U_T[k]")
model.addConstrs((quicksum(x[j, k] for k in K) <= config['U_L'][j] for j in J), "(3.3) The max resource sum for facility j is U_L[j]")
model.addConstrs((aX[j] <= config['M'] * y[j] for j in J), "(3.4) Extra Attractiveness Limit")

model.addConstrs((u[i, j] == (quicksum(config['V'][j][k] * x[j, k] for k in K) + aX[j]) / (config['D'][i][j]**2) for i in I for j in J), "(3.5) utility for facility j to customer i")
model.addConstrs((P[i, j] == u[i, j] / TA[i] for i in I for j in J), "(3.8) calculate P by uij/TAi")
## TA的限制式, 因為不能讓 G_exp中 addGenConstrExp內的變數為多個變數的線性組合所以要另外創TA變數
model.addConstrs((TA[i] == quicksum(u[i, j] for j in J) + quicksum(config['A_opponent_bar'][l] / (config['D_comp'][i][l]**2) for l in L) for i in I), "(3.7) calculate total TAi by u and A_bar")

## G的計算
# def G(tai):
#     return 1 - np.exp(-config['lambda_G'] * tai)
model.addConstrs((TAi_lambda[i] == -config['lambda_G'] * TA[i] for i in I), "negative tai*lambda constr")
model.addConstrs((P_TA_ratio[i] * TA[i] == P[i] for i in I), "tai*lambda constr") #(3.10)的右半側
### Gurobi無法直接在目標式有exponential
for i in I:
    model.addGenConstrExp(TAi_lambda[i], G_exp_vars[i], name=f"G_exp_constr_{i}")


model.update()
# Objective function: Maximize profit by attracting customers - costs
objective = quicksum(config['H'][i] * (1 - G_exp_vars[i]) * P_TA_ratio[i] for i in I) \
          - quicksum(config['F'][j] * y[j] + config['C'][j] * aX[j] + quicksum(config['B'][j][k] * x[j, k] for k in K) for j in J)
model.setObjective(objective, GRB.MAXIMIZE)

# Solve and Output
model.optimize()
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
else:
    print("No optimal solution found.")


In [ ]:
for var in model.getVars():
    print(f"{var.varName} = {var.x}")
print("")

# References

## G calculation substitude exponential
https://www.gurobi.com/documentation/10.0/refman/py_model_agc_exp.html


https://support.gurobi.com/hc/en-us/community/posts/360077178491-Build-an-objective-function-with-Log-and-Exponential-inside-


https://support.gurobi.com/hc/en-us/community/posts/10481250723217-How-to-add-log-and-exponential-term-in-objective-function